In [1]:
import pandas as pd

In [2]:
#year = 2005
#path = "sample_orig_"
#data_value = pd.read_table(path+str(year)+".txt", delimiter = "|", header = None)

data_value = pd.read_table("historical_data1_Q12007.txt", delimiter = "|", header = None)

##### As we can see, 25th row's data type is float64, so we can know its columns name is "Pre-HARP LOAN SEQUENCE NUMBER".

In [3]:
columns_header = ["CREDIT_SCORE","FIRST_PAYMENT_DATE","FIRST_TIME_HOMEBUYER_FLAG","MATURITY_DATE","MSA","MORTGAGE_INSURANCE_PERCENTAGE","NUMBER_OF_UNITS","OCCUPANCY_STATUS","ORIGINAL_COMBINED_LOAN_TO_VALUE","ORIGINAL_DEBT_TO_INCOME_RATIO","ORIGINAL_UPB","ORIGINAL_LOAN_TO_VALUE","ORIGINAL_INTEREST_RATE","CHANNEL","PREPAYMENT_PENALTY_MORTGAGE_FLAG","PRODUCT_TYPE","PROPERTY_STATE","PROPERTY_TYPE","POSTAL_CODE","LOAN_SEQUENCE_NUMBER","LOAN_PURPOSE","ORIGINAL_LOAN_TERM","NUMBER_OF_BORROWERS","SELLER_NAME","SERVICER_NAME","Pre_HARP_LOAN_SEQUENCE_NUMBER"]
#need to be a function
dataset = pd.read_table("sample_orig_2005.txt", header=None, delimiter = "|", names = columns_header)

In [4]:
dataset.describe()

,CREDIT_SCORE,FIRST_PAYMENT_DATE,MATURITY_DATE,MSA,MORTGAGE_INSURANCE_PERCENTAGE,NUMBER_OF_UNITS,ORIGINAL_COMBINED_LOAN_TO_VALUE,ORIGINAL_DEBT_TO_INCOME_RATIO,ORIGINAL_UPB,ORIGINAL_LOAN_TO_VALUE,ORIGINAL_INTEREST_RATE,POSTAL_CODE,ORIGINAL_LOAN_TERM,NUMBER_OF_BORROWERS,Pre_HARP_LOAN_SEQUENCE_NUMBER
count,50000.000000,50000.000000,50000.000000,42100.000000,50000.00000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,49999.000000,50000.000000,50000.000000,0.0
mean,728.499040,200522.933780,203206.171460,30611.265416,4.38436,1.026440,71.216100,65.38164,170691.860000,69.568040,5.805862,51021.006420,322.951280,1.614120,NaN
std,211.417207,36.458864,593.417833,11364.637919,34.14059,0.203523,21.022792,167.98203,86001.659492,20.219167,0.392373,29279.743726,71.009766,1.862548,NaN
min,300.000000,200502.000000,201003.000000,10180.000000,0.00000,1.000000,6.000000,1.00000,10000.000000,6.000000,4.250000,600.000000,60.000000,1.000000,NaN
25%,683.000000,200506.000000,203502.000000,19740.000000,0.00000,1.000000,60.000000,27.00000,104000.000000,60.000000,5.500000,28300.000000,360.000000,1.000000,NaN
50%,731.000000,200509.000000,203506.000000,33124.000000,0.00000,1.000000,76.000000,36.00000,154000.000000,75.000000,5.750000,48300.000000,360.000000,2.000000,NaN
75%,772.000000,200512.000000,203510.000000,40060.000000,0.00000,1.000000,80.000000,45.00000,224000.000000,80.000000,6.000000,78200.000000,360.000000,2.000000,NaN
max,9999.000000,201509.000000,205310.000000,49740.000000,999.00000,4.000000,999.000000,999.00000,692000.000000,999.000000,9.990000,99900.000000,576.000000,99.000000,NaN


In [5]:
import pandas_profiling

pandas_profiling.ProfileReport(dataset)

Number of variables,26
Number of observations,50000
Total Missing (%),4.5%
Total size in memory,9.9 MiB
Average record size in memory,208.0 B
Numeric,12
Categorical,9
Boolean,0
Date,0
Text (Unique),1
Rejected,4


##### Drop columns which NaN value is more than 70%.

In [6]:
def dropCol(dataset, cutoff = 0.3):
    for col_name in columns_header:
        n = len(dataset)
        value_count = dataset[col_name].count()
        if(value_count/n) < cutoff:
            dataset.drop(col_name, axis = 1, inplace = True)

In [7]:
dropCol(dataset, cutoff = 0.3)

##### Transfer data type.

In [8]:
def toDatetime(dataset):
    dataset["FIRST_PAYMENT_DATE"] = pd.to_datetime(dataset["FIRST_PAYMENT_DATE"], format = "%Y%m")
    dataset["MATURITY_DATE"] = pd.to_datetime(dataset["MATURITY_DATE"], format = "%Y%m")
    dataset['FIRST_PAYMENT_Year'] = pd.DatetimeIndex(dataset["FIRST_PAYMENT_DATE"]).year
    dataset['FIRST_PAYMENT_Month'] = pd.DatetimeIndex(dataset["FIRST_PAYMENT_DATE"]).month
    dataset['MATURITY_DATE_Year'] = pd.DatetimeIndex(dataset["MATURITY_DATE"]).year
    dataset['MATURITY_DATE_Month'] = pd.DatetimeIndex(dataset["MATURITY_DATE"]).month
    dataset.drop(["FIRST_PAYMENT_DATE","MATURITY_DATE"], axis = 1, inplace = True)

In [9]:
toDatetime(dataset)

##### Remove all not available values in the dataset like "9999" or "9"  
##### For numeric value, to replace by NaN, for object/string value, to replace by "O" which means other.

In [10]:
def replace_NaN(dataset):
    dataset["CREDIT_SCORE"].replace(9999, float('nan'), inplace = True)
    dataset["MORTGAGE_INSURANCE_PERCENTAGE"].replace(999, float('nan'), inplace = True)
    dataset["NUMBER_OF_UNITS"].replace(99, float('nan'), inplace = True)
    dataset["ORIGINAL_COMBINED_LOAN_TO_VALUE"].replace(999, float('nan'), inplace = True)
    dataset["ORIGINAL_DEBT_TO_INCOME_RATIO"].replace(999, float('nan'), inplace = True)
    dataset["ORIGINAL_LOAN_TO_VALUE"].replace(999, float('nan'), inplace = True)
    dataset["NUMBER_OF_BORROWERS"].replace(99, float('nan'), inplace = True)

def replace_O(dataset):
    dataset["OCCUPANCY_STATUS"].replace("9", "O", inplace = True)
    dataset["CHANNEL"].replace("9", "O", inplace = True)
    dataset["PROPERTY_TYPE"].replace("99", "O", inplace = True)
    dataset["LOAN_PURPOSE"].replace("9", "O", inplace = True)

In [11]:
replace_NaN(dataset)
replace_O(dataset)

##### Fill NaN with mean value.

In [12]:
def fillNaN(dataset):
    dataset.fillna(dataset.mean(), inplace =True)

In [13]:
fillNaN(dataset)

##### Transfer values in flag columns to 1 or 0.

In [14]:
def flag_toValue(dataset):
    dataset["FIRST_TIME_HOMEBUYER_FLAG"].replace("Y", 1, inplace = True)
    dataset["FIRST_TIME_HOMEBUYER_FLAG"].replace("N", 0, inplace = True)
    dataset["PREPAYMENT_PENALTY_MORTGAGE_FLAG"].replace("Y", 1, inplace = True)
    dataset["PREPAYMENT_PENALTY_MORTGAGE_FLAG"].replace("N", 0, inplace = True)
    
    ppmList = dataset["PREPAYMENT_PENALTY_MORTGAGE_FLAG"].tolist()
    p = ppmList.count(1)
    q = ppmList.count(0)
    if(p>q):
        dataset["PREPAYMENT_PENALTY_MORTGAGE_FLAG"].fillna(1, inplace = True)
    else:
        dataset["PREPAYMENT_PENALTY_MORTGAGE_FLAG"].fillna(0, inplace = True)
    
    firstList = dataset["FIRST_TIME_HOMEBUYER_FLAG"].tolist()
    m = ppmList.count(1)
    n = ppmList.count(0)
    if(m>n):
        dataset["FIRST_TIME_HOMEBUYER_FLAG"].replace("9", 1, inplace = True)
        dataset["FIRST_TIME_HOMEBUYER_FLAG"].fillna(1, inplace = True)
    else:
        dataset["FIRST_TIME_HOMEBUYER_FLAG"].replace("9", 0, inplace = True)
        dataset["FIRST_TIME_HOMEBUYER_FLAG"].fillna(0, inplace = True)

In [15]:
flag_toValue(dataset)

##### Because there is only one kind of value in product_type, so we can remove this column directly.

In [16]:
def drop_productType(dataset):
    dataset.drop("PRODUCT_TYPE", axis = 1, inplace = True)

In [17]:
drop_productType(dataset)

In [18]:
dataset.describe()

,CREDIT_SCORE,FIRST_TIME_HOMEBUYER_FLAG,MSA,MORTGAGE_INSURANCE_PERCENTAGE,NUMBER_OF_UNITS,ORIGINAL_COMBINED_LOAN_TO_VALUE,ORIGINAL_DEBT_TO_INCOME_RATIO,ORIGINAL_UPB,ORIGINAL_LOAN_TO_VALUE,ORIGINAL_INTEREST_RATE,PREPAYMENT_PENALTY_MORTGAGE_FLAG,POSTAL_CODE,ORIGINAL_LOAN_TERM,NUMBER_OF_BORROWERS,FIRST_PAYMENT_Year,FIRST_PAYMENT_Month,MATURITY_DATE_Year,MATURITY_DATE_Month
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,724.047063,0.082820,30611.265416,3.289078,1.026440,71.104753,35.314657,170691.860000,69.456495,5.805862,0.002700,51021.006420,322.951280,1.580998,2005.162380,6.695780,2031.995180,6.653460
std,58.517938,0.275613,10428.235597,8.727740,0.203523,18.402429,12.096033,86001.659492,17.468227,0.392373,0.051892,29279.450921,71.009766,0.493317,0.384572,3.430916,5.933166,3.382313
min,300.000000,0.000000,10180.000000,0.000000,1.000000,6.000000,1.000000,10000.000000,6.000000,4.250000,0.000000,600.000000,60.000000,1.000000,2005.000000,1.000000,2010.000000,1.000000
25%,683.000000,0.000000,22220.000000,0.000000,1.000000,60.000000,27.000000,104000.000000,60.000000,5.500000,0.000000,28300.000000,360.000000,1.000000,2005.000000,4.000000,2035.000000,4.000000
50%,730.000000,0.000000,30611.265416,0.000000,1.000000,76.000000,35.314657,154000.000000,75.000000,5.750000,0.000000,48300.000000,360.000000,2.000000,2005.000000,7.000000,2035.000000,7.000000
75%,772.000000,0.000000,38860.000000,0.000000,1.000000,80.000000,44.000000,224000.000000,80.000000,6.000000,0.000000,78200.000000,360.000000,2.000000,2005.000000,10.000000,2035.000000,10.000000
max,842.000000,1.000000,49740.000000,40.000000,4.000000,179.000000,65.000000,692000.000000,100.000000,9.990000,1.000000,99900.000000,576.000000,2.000000,2015.000000,12.000000,2053.000000,12.000000


In [19]:
dataset.head(100)

,CREDIT_SCORE,FIRST_TIME_HOMEBUYER_FLAG,MSA,MORTGAGE_INSURANCE_PERCENTAGE,NUMBER_OF_UNITS,OCCUPANCY_STATUS,ORIGINAL_COMBINED_LOAN_TO_VALUE,ORIGINAL_DEBT_TO_INCOME_RATIO,ORIGINAL_UPB,ORIGINAL_LOAN_TO_VALUE,...,LOAN_SEQUENCE_NUMBER,LOAN_PURPOSE,ORIGINAL_LOAN_TERM,NUMBER_OF_BORROWERS,SELLER_NAME,SERVICER_NAME,FIRST_PAYMENT_Year,FIRST_PAYMENT_Month,MATURITY_DATE_Year,MATURITY_DATE_Month
0,722.0,0,30611.265416,0.0,1,P,80.0,48.000000,62000,80.0,...,F105Q1000064,P,360,2.0,Other sellers,Other servicers,2005,4,2035,3
1,759.0,0,30611.265416,0.0,1,P,25.0,25.000000,198000,25.0,...,F105Q1000076,N,360,2.0,Other sellers,"PNCMTGESERVICES,INC",2005,3,2035,2
2,591.0,0,39100.000000,0.0,1,P,48.0,34.000000,100000,48.0,...,F105Q1000087,C,360,2.0,Other sellers,Other servicers,2005,4,2035,3
3,792.0,0,39100.000000,0.0,1,P,90.0,33.000000,334000,80.0,...,F105Q1000130,P,360,2.0,Other sellers,Other servicers,2005,3,2035,2
4,725.0,0,48864.000000,0.0,1,P,49.0,41.000000,125000,49.0,...,F105Q1000195,N,360,2.0,Other sellers,USBANKNA,2005,3,2035,2
5,788.0,0,30611.265416,0.0,1,P,75.0,55.000000,145000,75.0,...,F105Q1000217,N,360,2.0,Other sellers,"WELLSFARGOBANK,NA",2005,3,2035,2
6,691.0,0,30611.265416,35.0,1,P,97.0,48.000000,73000,97.0,...,F105Q1000227,P,360,2.0,Other sellers,Other servicers,2005,3,2035,2
7,687.0,0,30611.265416,0.0,1,P,40.0,46.000000,61000,40.0,...,F105Q1000261,P,360,2.0,Other sellers,"WELLSFARGOBANK,NA",2005,3,2035,2
8,784.0,0,30611.265416,0.0,1,P,35.0,30.000000,80000,35.0,...,F105Q1000282,C,360,2.0,Other sellers,Other servicers,2005,3,2035,2
9,703.0,0,30611.265416,0.0,1,P,67.0,53.000000,100000,67.0,...,F105Q1000305,P,360,2.0,Other sellers,Other servicers,2005,3,2035,2


##### Save dataframe to csv.file

In [20]:
#dataset.to_csv(path+str(year)+"_afterCleaning.csv", index = False)
dataset.to_csv("historical_data1_Q12007_after_cleaning.csv", index = False)